# fashion MNIST  
---  
## Tensorboard  
https://www.youtube.com/watch?v=pSexXMdruFM&list=PLZbbT5o_s2xrfNyHZsM6ufI0iZENK9xgG&index=29

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
# from torchvision import transforms

torch.set_printoptions(linewidth=120) # display option for output
torch.set_grad_enabled(True) # gradient tracking turned on (default)

from torch.utils.tensorboard import SummaryWriter # pip install future

In [2]:
print(torch.__version__)
print(torchvision.__version__)

1.2.0
0.4.0


In [3]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [4]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        # linear layer == fully connected layer == fc == dense layer
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # (1) input layer:
        # t = t
        
        # (2) hidden conv layer:
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (3) hidden conv layer:
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer:
        t = t.reshape(-1, 12*4*4)
        # t = t.flatten()
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer:
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer:
        t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        return t

In [5]:
train_set = torchvision.datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

---  
## Training loop  

In [ ]:
!tensorboard --logdir=runs

In [8]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(10):
    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch

        preds = network(images)
        loss = F.cross_entropy(preds, labels) # calculating the loss function

        optimizer.zero_grad() # zero out the gradients, because pytorch is actually adding the grads
        loss.backward() # calculating the gradients
        optimizer.step() # update the weight

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', 100*total_correct/len(train_set), epoch)

    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)

    print(f'epoch: {epoch} total correct: {total_correct} loss: {total_loss:.2f} accuracy {100*total_correct / len(train_set):.2f}%')

tb.close()

epoch: 0 total correct: 46007 loss: 366.88 accuracy 76.68%
epoch: 1 total correct: 50571 loss: 258.31 accuracy 84.28%
epoch: 2 total correct: 51389 loss: 236.03 accuracy 85.65%
epoch: 3 total correct: 51524 loss: 228.56 accuracy 85.87%
epoch: 4 total correct: 51938 loss: 218.57 accuracy 86.56%
epoch: 5 total correct: 52000 loss: 215.81 accuracy 86.67%
epoch: 6 total correct: 52133 loss: 211.46 accuracy 86.89%
epoch: 7 total correct: 52273 loss: 209.88 accuracy 87.12%
epoch: 8 total correct: 52141 loss: 214.13 accuracy 86.90%
epoch: 9 total correct: 52380 loss: 206.02 accuracy 87.30%
